In [1]:
import dgl
import torch as th

Using backend: pytorch


In [2]:
u, v = th.tensor([0, 0, 0, 1]), th.tensor([1, 2, 3, 3])

In [3]:
g = dgl.graph((u, v))

In [4]:
g

Graph(num_nodes=4, num_edges=4,
      ndata_schemes={}
      edata_schemes={})

In [5]:
g.nodes()

tensor([0, 1, 2, 3])

In [6]:
g.edges()

(tensor([0, 0, 0, 1]), tensor([1, 2, 3, 3]))

In [7]:
g.edges(form='all')

(tensor([0, 0, 0, 1]), tensor([1, 2, 3, 3]), tensor([0, 1, 2, 3]))

In [9]:
g

Graph(num_nodes=4, num_edges=4,
      ndata_schemes={}
      edata_schemes={})

In [10]:
graph_data = {
   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
   ('drug', 'interacts', 'gene'): (th.tensor([0, 1]), th.tensor([2, 3])),
   ('drug', 'treats', 'disease'): (th.tensor([1]), th.tensor([2]))
}
g = dgl.heterograph(graph_data)

In [11]:
g = dgl.heterograph(graph_data)

In [12]:
g

Graph(num_nodes={'disease': 3, 'drug': 3, 'gene': 4},
      num_edges={('drug', 'interacts', 'drug'): 2, ('drug', 'interacts', 'gene'): 2, ('drug', 'treats', 'disease'): 1},
      metagraph=[('drug', 'drug', 'interacts'), ('drug', 'gene', 'interacts'), ('drug', 'disease', 'treats')])

In [14]:
g.ntypes

['disease', 'drug', 'gene']

In [15]:
g.etypes

['interacts', 'interacts', 'treats']

In [16]:
g.canonical_etypes

[('drug', 'interacts', 'drug'),
 ('drug', 'interacts', 'gene'),
 ('drug', 'treats', 'disease')]

In [18]:
g.nodes('drug')

tensor([0, 1, 2])

In [2]:
dataset = dgl.data.CoraGraphDataset()
g = dataset[0]

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [3]:
g

Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={'feat': Scheme(shape=(1433,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})

In [ ]:
# 消息函数  接收edges  (src, dst, data)
# 聚合函数  接受nodes  (mailbox) sum max min
# 更新函数  接受nodes  此函数对 聚合函数 的聚合结果进行操作， 通常在消息传递的最后一步将其与节点的特征相结合，并将输出作为节点的新特征。

In [ ]:
def message_func(edges):
     return {'he': edges.src['hu'] + edges.dst['hv']}

In [6]:
g.edges()

(tensor([   0,    0,    0,  ..., 2707, 2707, 2707]),
 tensor([ 633, 1862, 2582,  ...,  598, 1473, 2706]))

In [7]:
import dgl.function as fn

In [25]:
g.apply_edges(fn.u_add_v('el', 'el', 'e'))

In [14]:
g

Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={'feat': Scheme(shape=(1433,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})

In [27]:
g.ndata['el'] = torch.ones(g.num_nodes())
g.ndata['er'] = torch.ones(g.num_nodes())*2


In [37]:
g.edata

{'e1': tensor([2., 2., 2.,  ..., 2., 2., 2.]), 'e2': tensor([3., 3., 3.,  ..., 3., 3., 3.]), 'e3': tensor([4., 4., 4.,  ..., 4., 4., 4.])}

In [53]:
g.ndata

{'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'el': tensor([1., 1., 1.,  ..., 1., 1., 1.]), 'er': tensor([2., 2., 2.,  ..., 2., 2., 2.])}

In [34]:
g.apply_edges(fn.u_add_v('el', 'el', 'e1'))
g.apply_edges(fn.u_add_v('el', 'er', 'e2'))
g.apply_edges(fn.u_add_v('er', 'er', 'e3'))

In [57]:
g.update_all(fn.u_mul_e('el', 'e3', 'm'), fn.sum('m', 'el'))

In [58]:
g.ndata['el']

tensor([ 160.,  128., 1280.,  ...,   16.,  224.,  720.])

In [59]:
g.apply_edges()

TypeError: apply_edges() missing 1 required positional argument: 'func'

In [74]:
g.ndata['feat'].shape

torch.Size([2708, 1433])

In [75]:
linear_src = torch.nn.Parameter(torch.FloatTensor(size=(1433, 16)))
linear_dst = torch.nn.Parameter(torch.FloatTensor(size=(1433, 16)))

In [81]:
out_src = g.ndata['feat'] @ linear_src
out_dst = g.ndata['feat'] @ linear_dst

In [90]:
g.srcdata['out_src'] = out_src
g.dstdata['out_dst'] = out_dst

In [91]:
g.apply_edges(fn.u_add_v('out_src', 'out_dst', 'out'))

In [94]:
g.edata['out'].shape

torch.Size([10556, 16])